In [ ]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
import json

## 1: Create Choropleth for Price/Sqm

1.1: read the csv of our rental prices and put it into a dataframe

In [ ]:
rentals = pd.read_csv('pararius_scraped.csv')
df = pd.DataFrame(rentals, columns = ['Address', 'Postcode', 'Size in m²', 'Rooms', 'Prize in €', 'Price/Sqm', 'Latitude', 'Longitude'])

1.2: To create a Choropleth, a geojson file will be used that assigns locations to postcodes. The latitude/longitude values from the dataset cannot be used directly. In this case, the postcodes are just the numbers. Therefore, we split the postcode column of our dataframe and create a new column that contains just the numbers.

In [ ]:
postcodes = pd.DataFrame()
postcodes[['Number', 'Letter']] = df.Postcode.str.split(expand=True)
df['Postcode_nr'] = postcodes['Number']
df.head()

1.3: Creating a choropleth: First, an empty map 'm' is created. Then the choropleth is added to it. The openpostcodevlakkenpc4.geojson file is the basis for assigning locations (areas) to the postcodes. 

In [ ]:
m = folium.Map(location=[52.370216, 4.895168], tiles='CartoDB Positron', zoom_start=12, control_scale=True)

folium.Choropleth(
    geo_data='openpostcodevlakkenpc4.geojson',
    name="Rental Price",
    data=df,
    columns=["Postcode_nr", "Price/Sqm"],
    key_on="properties.pc4",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    nan_fill_opacity=0,
    line_opacity=0.2,
    legend_name="Rental Price per m²",
).add_to(m)


1.4: Add a minimap in the lower right corner and a "Fullscreen" option to the map.

In [ ]:
minimap = plugins.MiniMap(toggle_display=True)
m.add_child(minimap)
plugins.Fullscreen(position='topright').add_to(m)

## 2: Add another layer that shows tram/metro stations with markers

2.1: Create a dataframe from a csv containing the stations and latitude/longitude data of each station

In [ ]:
tram = pd.read_csv('Tram_metro_stops.csv')
tram_df = pd.DataFrame(tram)
tram_df.rename(columns={'Unnamed: 0': 'Station Tram/Metro'}, inplace=True)
tram_df.head()

2.2: Create a FeatureGroup 'a' which can be optionally overlayed on the basis map. For every station add a marker to 'a'. Finally, add 'a' to our basis map 'm'

In [ ]:
from folium.plugins import MarkerCluster

a = folium.FeatureGroup(name='Stations', overlay=True, show=False)

for i in range(0,len(tram_df)):
   folium.Marker(
      location=[tram_df.iloc[i]['lat'], tram_df.iloc[i]['lon']],
      popup=tram_df.iloc[i]['Station Tram/Metro'],
      icon=folium.Icon(color='red', icon_color="white", icon='subway', prefix='fa')
   ).add_to(a)

m.add_child(a)


## 3: Add another layer that shows break ins as a heatmap

3.1: Create a dataframe from csv containing the break ins and longitude/latitude data for each break in

In [ ]:
breaks = pd.read_csv('AMS_break_ins_per_postal_code.csv')
break_ins = pd.DataFrame(breaks)
break_ins.head()

3.2: Create a list 'l' of all latitude/longitude pairs

In [ ]:
l = list()
for i in range(0,len(break_ins)):
    x = (break_ins.iloc[i]['Latitude'],break_ins.iloc[i]['Longitude'])
    l.append(x)

3.3: Create a heatmap from all items in list 'l' and add to base map 'm'. Add a Layer control to 'm' so we can choose whether the Stations & Break Ins layers are shown via a menu that is displayed on the map. Show 'm'. 

In [ ]:
from folium.plugins import HeatMap

In [ ]:
HeatMap(l, name = 'Break Ins', overlay=True, show=False).add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
#m.save('/Users/...path.html')